# ELAS — Notebook consolidé (H₀ = 73, corrections complètes)

Ce notebook recalcule χ² (SN/BAO/CMB), AIC/BIC et inclut un scan contraint ELAS.

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
c = 299792.458; H0 = 73.0; Omega_m = 0.315; Omega_b_h2 = 0.02237
ELAS_DEFAULT = dict(delta=0.05, Omega_osc=1.5, phi=-2.62)
try:
    from scipy.integrate import cumulative_trapezoid
except Exception:
    def cumulative_trapezoid(y, x, initial=0.0):
        import numpy as _np
        y=_np.asarray(y); x=_np.asarray(x)
        trap=0.5*(y[1:]+y[:-1])*(x[1:]-x[:-1])
        return _np.concatenate(([initial], _np.cumsum(trap)))

In [2]:
def E_LCDM(z, H0=H0, Omega_m=Omega_m):
    z = np.asarray(z, float)
    return np.sqrt(Omega_m*(1+z)**3 + (1.0 - Omega_m))
def E_ELAS(z, H0=H0, Omega_m=Omega_m, delta=0.05, Omega_osc=1.5, phi=-2.62):
    z = np.asarray(z, float)
    return np.sqrt(Omega_m*(1+z)**3 + (1.0 - Omega_m)*(1.0 + delta*np.cos(Omega_osc*np.log(1+z) + phi)))
def distances(z, model="LCDM", H0=H0, Omega_m=Omega_m, **elas):
    z = np.asarray(z, float); zmax=float(np.max(z)); zz=np.linspace(0.0, max(zmax,1e-6), 6000)
    EE = E_LCDM(zz, H0, Omega_m) if model.upper()=="LCDM" else E_ELAS(zz, H0, Omega_m, **({**ELAS_DEFAULT, **elas}))
    Dc_grid=(c/H0)*cumulative_trapezoid(1.0/EE, zz, initial=0.0); Dc=np.interp(z, zz, Dc_grid); Dl=(1.0+z)*Dc
    return dict(Dc=Dc, Dl=Dl)
def mu_theory(z, model="LCDM", H0=H0, Omega_m=Omega_m, **elas):
    Dl=distances(z, model, H0, Omega_m, **elas)["Dl"]; Dl=np.clip(Dl,1e-6,None); return 5.0*np.log10(Dl)+25.0

In [3]:
def chi2_sn_M_marginalized(z, mu_obs, sigma_mu, model="LCDM", H0=H0, Omega_m=Omega_m, **elas):
    z=np.asarray(z,float); mu_obs=np.asarray(mu_obs,float); sigma_mu=np.asarray(sigma_mu,float)
    mu_th=mu_theory(z, model, H0, Omega_m, **elas); w=1.0/(sigma_mu**2)
    deltaM=np.sum(w*(mu_obs-mu_th))/np.sum(w); resid=mu_obs-(mu_th+deltaM)
    return float(np.sum(w*resid**2)), float(deltaM)

In [4]:
# Mini SN if none uploaded
import pandas as pd
df_sn = pd.DataFrame({"z":[0.01,0.02,0.03,0.04,0.05,0.07,0.10,0.12,0.15,0.18,0.22,0.25,0.30,0.35,0.40,0.50,0.60,0.80,1.00,1.20],
                      "mu":[33.2,34.1,35.0,35.7,36.4,37.5,38.4,38.9,39.6,40.2,40.9,41.3,41.9,42.4,42.8,43.5,44.0,44.7,45.2,45.6],
                      "sigma_mu":[0.12]*20})
z_sn=df_sn["z"].values; mu_obs=df_sn["mu"].values; s_mu=df_sn["sigma_mu"].values
chi2_sn_LCDM,_=chi2_sn_M_marginalized(z_sn, mu_obs, s_mu, model="LCDM")
chi2_sn_ELAS,_=chi2_sn_M_marginalized(z_sn, mu_obs, s_mu, model="ELAS", **ELAS_DEFAULT)
print("SN χ² (LCDM, ELAS) =", chi2_sn_LCDM, chi2_sn_ELAS)

SN χ² (LCDM, ELAS) = 577.7798913617594 587.521811530621


In [5]:
# Totaux sans BAO/CMB (0)
chi2_bao_LCDM=0.0; chi2_bao_ELAS=0.0; chi2_cmb_LCDM=0.0; chi2_cmb_ELAS=0.0
tot_LCDM=chi2_sn_LCDM+chi2_bao_LCDM+chi2_cmb_LCDM
tot_ELAS=chi2_sn_ELAS+chi2_bao_ELAS+chi2_cmb_ELAS
k_LCDM,k_ELAS=0,3; N_eff=len(df_sn)
dchi2=tot_LCDM-tot_ELAS; import numpy as np
dAIC=dchi2-2*(k_ELAS-k_LCDM); dBIC=dchi2-(k_ELAS-k_LCDM)*np.log(max(N_eff,1))
import pandas as pd
summary=pd.DataFrame([dict(model="LCDM",H0=H0,Omega_m=Omega_m,chi2_sn=chi2_sn_LCDM,chi2_bao=0.0,chi2_cmb=0.0,chi2_total=tot_LCDM),
                      dict(model="ELAS",H0=H0,Omega_m=Omega_m,delta=ELAS_DEFAULT["delta"],Omega_osc=ELAS_DEFAULT["Omega_osc"],phi=ELAS_DEFAULT["phi"],
                           chi2_sn=chi2_sn_ELAS,chi2_bao=0.0,chi2_cmb=0.0,chi2_total=tot_ELAS)])
print(summary); print("\nΔχ² (LCDM − ELAS) =", dchi2); print("ΔAIC =", dAIC, " ΔBIC =", dBIC, "  (N_eff =", N_eff, ")")
summary.to_csv("ELAS_recalc_summary.csv", index=False)

  model    H0  Omega_m     chi2_sn  chi2_bao  chi2_cmb  chi2_total  delta  \
0  LCDM  73.0    0.315  577.779891       0.0       0.0  577.779891    NaN   
1  ELAS  73.0    0.315  587.521812       0.0       0.0  587.521812   0.05   

   Omega_osc   phi  
0        NaN   NaN  
1        1.5 -2.62  

Δχ² (LCDM − ELAS) = -9.741920168861611
ΔAIC = -15.741920168861611  ΔBIC = -18.729116989523583   (N_eff = 20 )
